save data of Grimm to individual csv

In [ ]:
import pandas as pd
import os

raw_data_file = "..\data\humu22768-sup-0001-suppmat\humu22768-sup-0002-data.xlsx"
df = pd.read_excel(raw_data_file, index_col=0, sheet_name = None, usecols=["CHR", "Nuc-Pos", "REF-Nuc", "ALT-Nuc"], engine="openpyxl")

variant_file_dir = "..\data\\variant"
if not os.path.exists(variant_file_dir):
    os.mkdir(variant_file_dir)
for key in df.keys():
        _ = df[key]
        _.to_csv(os.path.join(variant_file_dir, "{}.csv".format(key)))

convert csv to vcf for annotation

In [ ]:
# 把snp提取到vcf文件中
import pandas 
import os

variant_files_dir = "../data/variant"
output_dir = "../data/variant"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
print("starting")
for root, dirs, files in os.walk(variant_files_dir):
    for file_ in files:
        path = os.path.join(root, file_)
        print("process {}".format(path))
        df_variants = pandas.read_csv(path, index_col=0)
        print(list(df_variants))
        output_file = file_.split(".")[0] + ".vcf"
        if output_file is None:
            output_file = str(uuid.uuid4())
        output = os.path.join(output_dir, output_file)
        with open(output, "w+") as f:
            f.write("##fileformat=VCFv4.3\n")
            f.write("#CHROM\tPOS\tID\tREF\tALT\tQUAL\tFILTER\tINFO\n")
            for index, row in df_variants.iterrows():
                chr = index
                pos = row["Nuc-Pos"]
                ref = row["REF-Nuc"]
                alt = row["ALT-Nuc"]
                f.write("{}\t{}\t.\t{}\t{}\t.\t.\t.\n".format(chr, pos, ref, alt))

process the output of Annovar

In [1]:
from sklearn import preprocessing
chr_lbl = preprocessing.LabelEncoder()
chr_lbl.fit(['1', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '3',  '4', '5', '6', '7', '8', '9', 'X', 'Y'])

amid_acid_lbl= preprocessing.LabelEncoder()
amid_acid_lbl.fit(['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','U', 'V','W','X','Y'])

LabelEncoder()

In [2]:
def encode(row):
    chr_ = row['Chr']
    a_ref = row['A_Ref']
    a_alt = row['A_Alt']
    if chr_.startswith('chr'):
        chr_ = chr_[3:]
    try:
        chr_encoded = chr_lbl.transform([chr_])
    except ValueError:
        chr_encoded = [None]
    try:
        a_ref_encoded = amid_acid_lbl.transform([a_ref])
    except ValueError:
        a_ref_encoded = [None]
    try:
        a_alt_encoded = amid_acid_lbl.transform([a_alt])
    except ValueError:
        a_alt_encoded = [None]
    return chr_encoded[0], a_ref_encoded[0], a_alt_encoded[0]

In [ ]:
# 数据预处理
import pandas as pd
import os
import numpy as np
from sklearn import preprocessing

# 训练数据中包含的特征
features = ['True Label', 'Chr', 'Start', 'End', 'Ref', 'Alt', 'AAChange.refGene', 'SIFT_score', 'Polyphen2_HDIV_score', 'Polyphen2_HVAR_score', 'LRT_score', 'MutationTaster_score', 'MutationAssessor_score', 'FATHMM_score', 'RadialSVM_score', 'LR_score', 'VEST3_score', 'CADD_phred']
# 原始数据文件目录
raw_data_dir = '../data/variant/grimm_annovar_annotated'
flitered_data_output_dir = '../data/filter'
if not os.path.exists(flitered_data_output_dir):
    os.mkdir(flitered_data_output_dir)
# 合法染色体
chrs = []
for i in range(1, 23):
    chrs.append(str(i))
chrs.append("X")
chrs.append("Y")


# 从转录本提取氨基酸突变。eg LOXL4:NM_032211:exon3:c.C406A:p.R136S， aa_ref为R， aa_alt为S
def split_transcript(row):
    trans = row["AAChange.refGene"]
    trans = trans.split(",")
    # first
    tran = trans[0]
    aachange = tran.split(":")
    _ = aachange[-1].split(".")[-1]
    aa_ref = _[0]
    aa_alt = _[-1]
    return aa_ref, aa_alt


for dir_, _, filenames in os.walk(raw_data_dir):
    for filename in filenames:
        print("processing {}".format(filename))
        df = pd.read_csv(os.path.join(raw_data_dir, filename))
        # df = df[features]
        # .置空
        df = df[~(df == '.')]
        # # 剔除非法染色体
        df["Chr"] = df["Chr"].astype("str")
        if filename == "predictSNPSelected.hg19_multianno.csv" or filename == "SwissVarSelected.hg19_multianno.csv":
            df[["Chr"]] = df.apply(lambda x: x["Chr"][3:], axis=1)
        df = df[(df["Chr"].isin(chrs))]
        # 处理氨基酸突变序列
        df = df[~(df["AAChange.refGene"].isnull())]
        df[["A_Ref", "A_Alt"]] = df.apply(split_transcript, axis=1, result_type="expand")
        df = df.drop(["AAChange.refGene"], axis=1)
        # 类型转换
        df = df.astype(dtype={'True Label': object, "SIFT_score": float, "Polyphen2_HDIV_score": float, "Polyphen2_HVAR_score": float, "LRT_score": float, "MutationTaster_score":float, "MutationAssessor_score": float, "FATHMM_score":float, "RadialSVM_score":float, "LR_score": float, "VEST3_score":float, "CADD_phred":float})
        df[['Chr_', 'A_Ref_', 'A_Alt_']] = df.apply(encode, axis=1, result_type='expand')
        df["True Label"].replace(-1, 0, inplace=True)
        df.to_csv(os.path.join(flitered_data_output_dir, "{}".format(filename)), index=0)

save processed data

In [ ]:
# 去重
import pandas as pd

dfs = {}
input_dir = "../data/filter"
for root, _, files in os.walk(input_dir):
    for file_ in files:
        dataset = file_.split(".")[0]
        df = pd.read_csv(os.path.join(root, file_))
        dfs[dataset] = df

datasets = ['ExoVarFiltered', 'HumVarFiltered', 'predictSNPSelected', 'SwissVarSelected', 'VariBenchSelected']
df = pd.concat([dfs['HumVarFiltered'], dfs['predictSNPSelected'], dfs['VariBenchSelected']])
df.drop_duplicates(subset=['Chr', 'Start', 'End', 'Ref', 'Alt'], inplace=True)

df.to_csv('../data/dataset/training.csv', index=False)